## Step 0 – Imports and helper functions  
This section loads core scientific libraries (`pandas`, `numpy`, `pathlib`) and defines constants used throughout the data-processing pipeline.  
It includes:  
- The 95 % z-score for Wilson confidence intervals (`Z_95`)  
- A dictionary converting month names to numbers (`MONTH_MAP`)  
- A vectorized `wilson_ci()` function that computes the binomial proportion and 95 % confidence interval for survey detections.  
These utilities are reused later to calculate occupancy estimates for *Zostera capricorni* and *Halophila ovalis*.


In [21]:
# Step 0 — Imports & helpers
# If needed (run in a terminal/Anaconda prompt first):
# pip install pandas numpy matplotlib

import pandas as pd
import numpy as np
from pathlib import Path

# 95% z-score (for Wilson CI)
Z_95 = 1.959963984540054

# Month name → number (only used if MONTH is text like "October")
MONTH_MAP = {m.lower(): i for i, m in enumerate(
    ["January","February","March","April","May","June","July","August","September","October","November","December"], start=1)}

def wilson_ci(k, n, z=Z_95):
    """Vectorized Wilson 95% CI for binomial proportion."""
    k = np.asarray(k, dtype=float)
    n = np.asarray(n, dtype=float)
    p = np.divide(k, n, out=np.full_like(k, np.nan, dtype=float), where=n>0)
    denom = 1 + z**2/n
    center = (p + z**2/(2*n)) / denom
    half = (z / denom) * np.sqrt((p*(1-p)/n) + (z**2/(4*n**2)))
    return p, center - half, center + half

## Step 1 – File paths and data import  
This block defines the working directory and helper functions for locating and reading input files.  
It performs three main tasks:  
1. Uses `pick()` to automatically find the biology, environment, and site-location files based on name patterns.  
2. Implements `read_any()` to load either CSV/TXT or Excel files using the appropriate pandas reader.  
3. Verifies successful loading by printing file names and dataset dimensions for `bio`, `env`, and `sites_df`.  
This ensures all required inputs exist and are ready for processing.


In [22]:
from pathlib import Path
import pandas as pd


RAW = Path(".")         

if not RAW.exists():
    raise FileNotFoundError(f"{RAW.resolve()}")


def pick(pattern: str) -> Path:
    hits = sorted(RAW.glob(pattern))
    if not hits:
        raise FileNotFoundError(f" {RAW} {pattern}")
    return hits[0]

BIO_PATH   = pick("GBR_NESP-TWQ-3.2.1-5.4_JCU_Seagr*.*")
ENV_PATH   = pick("2509.28d55d4-collected*.*")
SITES_PATH = pick("seagrass location*.*")

print("Using files:\n -", BIO_PATH.name, "\n -", ENV_PATH.name, "\n -", SITES_PATH.name)


def read_any(p: Path):
    ext = p.suffix.lower()
    if ext in {".csv", ".txt"}:
        return pd.read_csv(p, low_memory=False, encoding="utf-8-sig")
    elif ext in {".xlsx", ".xls"}:
        return pd.read_excel(p)   
    else:
        raise ValueError(f"{ext}")


bio      = read_any(BIO_PATH)
env      = read_any(ENV_PATH)
sites_df = read_any(SITES_PATH)

print("Loaded → bio:", bio.shape, "| env:", env.shape, "| sites:", sites_df.shape)

Using files:
 - GBR_NESP-TWQ-3.2.1-5.4_JCU_Seagrass_1984-2018_Site-surveys.csv 
 - 2509.28d55d4-collected.csv 
 - seagrass location.csv
Loaded → bio: (81387, 24) | env: (5600, 12) | sites: (8, 3)


## Step 2 – Extract canonical site list  
This section reads the list of seagrass monitoring sites from the location file.  
It automatically detects the correct site-name column (e.g., `survey_nam`, `site`, or `location`) and extracts all unique names.  
Whitespace and formatting are cleaned to produce a standardized, alphabetically sorted `SITE_LIST`.  
This canonical list is used throughout the analysis to ensure site names are consistent between biology and environmental datasets.


In [23]:
# Site list (from your 8-site CSV)
# Pick the site-name column (your file looks like: name, LAT, LON)
site_col_guess = next((c for c in sites_df.columns
                       if str(c).lower() in {"survey_nam","site name","site","location","name"}),
                      sites_df.columns[0])

SITE_LIST = sorted(sites_df[site_col_guess].astype(str).str.strip().unique())
print("Sites:", SITE_LIST)

Sites: ['Abbot_Point', 'Cairns', 'Clairview', 'Gladstone', 'HayPoint', 'Mourilyan', 'OSRA', 'Townsville']


## Step 3 – Build monthly ZC occupancy (2010–2018)
Identify key columns in the biology table case-insensitively, then standardize site names to the canonical 8.  
Parse months robustly (numbers, full names, 3-letter) to construct a monthly `date` (YYYY-MM-01) and filter to 2010–2018.  
Create a `ZC_present` flag and aggregate by site×month to get n, k, p, Wilson 95% CI, and continuity-corrected `p_cc`.  
Print year coverage and site ranges to verify parsing; preview the first rows.


In [24]:
# ---- Rebuild monthly occupancy from raw `bio` (robust + keeps YEAR until the end) ----
import pandas as pd
import numpy as np

# 1) Identify key columns (case-insensitive)
colmap = {c.lower(): c for c in bio.columns}

def pick(opts):
    for o in opts:
        if o in colmap:
            return colmap[o]
    raise KeyError(f"None of {opts} found. Available: {list(bio.columns)[:20]} ...")

site_col  = pick(["survey_nam","site name","site","location"])
year_col  = pick(["year"])
month_col = pick(["month"])

# Zostera capricorni column (e.g., 'Z_CAPRICOR')
zcol = None
for c in bio.columns:
    cl = str(c).lower()
    if cl.startswith("z_") and ("cap" in cl or "mueller" in cl or "zostera" in cl):
        zcol = c; break
if zcol is None:
    zcol = pick(["z_capricor"])

# 2) Normalise site names to catch minor variants and keep your 8 sites
def _norm(s): return "".join(ch for ch in str(s).lower() if ch.isalnum())
site_norm_set = {_norm(s) for s in SITE_LIST}
site_map = { _norm(s): s for s in SITE_LIST }

bio_f = bio.copy()
bio_f["_site_norm"] = bio_f[site_col].apply(_norm)
bio_f = bio_f[bio_f["_site_norm"].isin(site_map)].copy()
bio_f["SURVEY_NAM"] = bio_f["_site_norm"].map(site_map)
bio_f.drop(columns=["_site_norm"], inplace=True)

# 3) Filter years, parse month robustly (numbers, full names, 3-letter abbr; trims spaces)
bio_f[year_col] = pd.to_numeric(bio_f[year_col], errors="coerce")
bio_f = bio_f[bio_f[year_col].between(2010, 2018)].copy()

mn = bio_f[month_col].astype(str).str.strip().str.lower()
month_num = pd.to_numeric(mn, errors="coerce")

# full names then 3-letter abbreviations
full_map = MONTH_MAP.copy()                    # from Step 0
abbr_map = {k[:3]: v for k, v in MONTH_MAP.items()}
mask = month_num.isna()
month_num[mask]  = mn[mask].map(full_map)
mask2 = month_num.isna()
month_num[mask2] = mn[mask2].str[:3].map(abbr_map)

bio_f["MONTH_NUM"] = pd.to_numeric(month_num, errors="coerce").astype("Int64")
bio_f["date"] = pd.to_datetime(
    dict(year=bio_f[year_col], month=bio_f["MONTH_NUM"], day=1),
    errors="coerce"
)

print("Year coverage after parsing:", sorted(bio_f[year_col].dropna().unique().tolist()))
print("Rows with unparsed dates (should be 0):", int(bio_f["date"].isna().sum()))
if bio_f["date"].isna().any():
    display(bio_f[bio_f["date"].isna()][[site_col, year_col, month_col]].head(10))

# 4) Z. capricorni flag and occupancy aggregation
zc = bio_f[zcol].astype(str).str.strip().str.lower()
bio_f["ZC_present"] = zc.isin(["yes","present","1","true","y"])

occ = (bio_f.groupby(["SURVEY_NAM","date"], as_index=False)
             .agg(n=("ZC_present","size"), k=("ZC_present","sum")))

# p and Wilson CI (clip CI to [0,1] to avoid tiny negative due to roundoff)
occ["p"], occ["p_lo"], occ["p_hi"] = wilson_ci(occ["k"], occ["n"])
occ["p_lo"] = occ["p_lo"].clip(lower=0); occ["p_hi"] = occ["p_hi"].clip(upper=1)
occ["p_cc"] = (occ["k"] + 0.5) / (occ["n"] + 1.0)

# Check years now
occ["year"] = occ["date"].dt.year
print("Occupancy years found:", sorted(occ["year"].unique().tolist()))
display(occ.groupby("SURVEY_NAM")["year"].agg(["min","max","nunique"]).sort_index())
occ = occ.drop(columns=["year"]).sort_values(["SURVEY_NAM","date"])

# Peek
occ.head(12)

Year coverage after parsing: [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
Rows with unparsed dates (should be 0): 0
Occupancy years found: [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


,min,max,nunique
SURVEY_NAM,,,
Abbot_Point,2010,2018,9
Cairns,2010,2018,9
Clairview,2017,2018,2
Gladstone,2010,2018,9
HayPoint,2011,2018,6
Mourilyan,2010,2018,9
OSRA,2011,2014,4
Townsville,2010,2018,9


,SURVEY_NAM,date,n,k,p,p_lo,p_hi,p_cc
0,Abbot_Point,2010-02-01,27,0,0.000000,0.000000,0.124555,0.017857
1,Abbot_Point,2010-06-01,158,7,0.044304,0.021624,0.088616,0.047170
2,Abbot_Point,2010-11-01,36,0,0.000000,0.000000,0.096419,0.013514
3,Abbot_Point,2010-12-01,137,5,0.036496,0.015688,0.082589,0.039855
4,Abbot_Point,2011-03-01,113,0,0.000000,0.000000,0.032878,0.004386
5,Abbot_Point,2011-05-01,109,0,0.000000,0.000000,0.034043,0.004545
6,Abbot_Point,2011-09-01,161,0,0.000000,0.000000,0.023304,0.003086
7,Abbot_Point,2012-02-01,138,0,0.000000,0.000000,0.027083,0.003597
8,Abbot_Point,2012-06-01,115,0,0.000000,0.000000,0.032324,0.004310
9,Abbot_Point,2012-09-01,117,0,0.000000,0.000000,0.031789,0.004237


## Step 4 – Quality check: sampling effort and detections by site  
This section summarizes sampling coverage and detection rates across sites.  
It calculates the number of distinct months and years sampled, total survey counts, and total *Zostera capricorni* detections.  
A secondary check counts how many years (2010–2018) each site recorded at least one detection.  
These metrics help identify uneven sampling effort or data gaps before environmental modeling.


In [25]:
# QC summary: sampling and detections by site
occ_qc = (
    occ.assign(year=occ["date"].dt.year)
       .groupby("SURVEY_NAM")
       .agg(
           months_sampled = ("date", "nunique"),
           years_sampled  = ("year", "nunique"),
           total_samples  = ("n", "sum"),
           total_Zc_hits  = ("k", "sum"),
       )
       .sort_index()
)
print(occ_qc)

# How many years (2010–2018) each site had at least one Zc detection?
yrs_with_zc = (
    occ.assign(year=occ["date"].dt.year)
       .groupby(["SURVEY_NAM","year"])["k"].sum()
       .gt(0).groupby("SURVEY_NAM").sum()
       .rename("years_with_Zc")
)
print(yrs_with_zc)

             months_sampled  years_sampled  total_samples  total_Zc_hits
SURVEY_NAM                                                              
Abbot_Point              26              9           3590             31
Cairns                   24              9           4250            185
Clairview                 3              2            245             50
Gladstone                17              9          23578           3447
HayPoint                  9              6           1914              5
Mourilyan                17              9           2056              0
OSRA                      4              4           2833              7
Townsville               12              9           6321            914
SURVEY_NAM
Abbot_Point    7
Cairns         9
Clairview      2
Gladstone      9
HayPoint       2
Mourilyan      0
OSRA           1
Townsville     9
Name: years_with_Zc, dtype: int64


## Step 5 – eReefs drivers: monthly panel and derived speeds  
Convert the raw eReefs table into a tidy monthly site×variable matrix.  
Standardize the datetime column, filter to the 8 sites and 2010–2018, then pivot `Variable` into wide columns of monthly means.  
Derive `current_speed = √(u²+v²)` and `wind_speed = √(wspeed_u²+wspeed_v²)` when vector components are available.  
Output is a driver panel keyed by `SURVEY_NAM` and `date` for merging with occupancy.


In [26]:
# Pivot eReefs drivers and derive speeds
env2 = env.copy()
date_col = "Aggregated Date/Time" if "Aggregated Date/Time" in env2.columns else "date"
env2["date"] = pd.to_datetime(env2[date_col], errors="coerce")

env2 = env2[env2["Site Name"].isin(occ["SURVEY_NAM"].unique())]
env2 = env2[env2["date"].dt.year.between(2010, 2018)]

env_w = (env2.pivot_table(index=["Site Name","date"], columns="Variable", values="mean", aggfunc="mean")
              .reset_index())
env_w.columns.name = None
env_w = env_w.rename(columns={"Site Name":"SURVEY_NAM"})

if {"u","v"}.issubset(env_w.columns):
    env_w["current_speed"] = np.sqrt(env_w["u"]**2 + env_w["v"]**2)
if {"wspeed_u","wspeed_v"}.issubset(env_w.columns):
    env_w["wind_speed"] = np.sqrt(env_w["wspeed_u"]**2 + env_w["wspeed_v"]**2)

## Step 6 – Merge occupancy with drivers; add seasonality and lags  
Join the ZC occupancy table with the eReefs driver panel on site and month.  
Encode seasonality via `sin12` and `cos12` using calendar month.  
Create 1–3 month lags for available drivers (`temp`, `salt`, `eta`, `current_speed`, `wind_speed`) by site.  
Add `p_cc_l1` to enable simple autoregressive baselines and SARIMAX exogenous setups.


In [27]:
# Merge + add seasonality and 1–3 month lags
merged = occ.merge(env_w, on=["SURVEY_NAM","date"], how="left")
merged["month"] = merged["date"].dt.month
merged["sin12"] = np.sin(2*np.pi*merged["month"]/12)
merged["cos12"] = np.cos(2*np.pi*merged["month"]/12)

driver_cols = [c for c in ["temp","salt","eta","current_speed","wind_speed"] if c in merged.columns]
for v in driver_cols:
    for L in (1,2,3):
        merged[f"{v}_l{L}"] = merged.groupby("SURVEY_NAM")[v].shift(L)

merged["p_cc_l1"] = merged.groupby("SURVEY_NAM")["p_cc"].shift(1)

## Step 7 – Save cleaned outputs for analysis  
Export the processed datasets to CSV for reuse in exploratory and modeling notebooks.  
Three outputs are generated:  
1. `Zc_monthly_occupancy_2010_2018.csv` – monthly ZC occupancy by site  
2. `env_drivers_pivot_2010_2018.csv` – monthly environmental drivers in wide format  
3. `Zc_occ_with_env_2010_2018.csv` – merged dataset with lags and seasonal terms  
All files are saved to the working directory to ensure reproducibility across analyses.


In [28]:
# Save tidy outputs (so you can use them in EDA/models)
from pathlib import Path
OUT = Path(".")
OUT.mkdir(exist_ok=True)

occ.to_csv(OUT / "Zc_monthly_occupancy_2010_2018.csv", index=False)
env_w.to_csv(OUT / "env_drivers_pivot_2010_2018.csv", index=False)
merged.to_csv(OUT / "Zc_occ_with_env_2010_2018.csv", index=False)
print("Saved to:", OUT)

Saved to: .


## Step 8 – HO pipeline: occupancy, drivers, lags, and exports (2010–2018)
Build *Halophila ovalis* monthly occupancy as a biotic covariate, aligned to the same sites and window as ZC.  
Load biology, environment, and site files; robustly detect columns; parse months; compute n, k, p, Wilson CI, and `p_cc`.  
Pivot eReefs (or keep wide), derive `current_speed` and `wind_speed`, merge with HO, add 1–3 month driver lags and `p_cc_l1`, then save:
`Ho_monthly_occupancy_2010_2018.csv` and `Ho_occ_with_env_2010_2018.csv`.


In [29]:
# --- HO pipeline (final): same outputs, robust site column, 2010–2018 filter ---
import pandas as pd
import numpy as np
from pathlib import Path
import re

# ---------------------- Paths (relative) ----------------------
BIO_PATH   = Path("GBR_NESP-TWQ-3.2.1-5.4_JCU_Seagrass_1984-2018_Site-surveys.csv")
ENV_PATH   = Path("2509.28d55d4-collected.csv")
SITES_PATH = Path("seagrass location.csv")
OUT        = Path(".")  # or Path("./processed")
OUT.mkdir(exist_ok=True)

YEAR_MIN, YEAR_MAX = 2010, 2018  # required time window

# ---------------------- Load ----------------------
bio      = pd.read_csv(BIO_PATH, low_memory=False)
env      = pd.read_csv(ENV_PATH)
sites_df = pd.read_csv(SITES_PATH)

# ---------------------- Helpers ----------------------
MONTH_MAP = {
    **{m.lower(): i for i, m in enumerate(
        ["January","February","March","April","May","June","July","August",
         "September","October","November","December"], start=1)},
    "jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12
}

def find_col(df, candidates, contains=None):
    """Return the first matching column name (original casing)."""
    cols = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c and c.lower() in cols:
            return cols[c.lower()]
    if contains:
        for c in df.columns:
            if contains.lower() in c.lower():
                return c
    return None

def parse_month_series(s):
    """Accept numeric, English names/abbrevs, or strings like '9月'."""
    s = s.astype(str).str.strip()
    out = pd.Series(np.nan, index=s.index, dtype="float")

    # pure digits
    mask_num = s.str.fullmatch(r"\d{1,2}")
    out.loc[mask_num] = s.loc[mask_num].astype(float)

    # '9月' or '09月'
    zh = s.str.extract(r"^\s*(\d{1,2})\s*月\s*$")[0]
    out.loc[zh.notna()] = zh.loc[zh.notna()].astype(float)

    # English names / 3-letter abbrevs
    rest = s[(~mask_num) & (zh.isna())]
    mapped = rest.str.lower().map(MONTH_MAP)
    out.loc[mapped.index] = mapped
    return out.astype("Int64")

Z_95 = 1.959963984540054
def wilson_ci(k, n, z=Z_95):
    k = np.asarray(k, dtype=float)
    n = np.asarray(n, dtype=float)
    p = np.divide(k, n, out=np.full_like(k, np.nan, dtype=float), where=n>0)
    denom  = 1 + z**2/n
    center = (p + z**2/(2*n)) / denom
    half   = (z * np.sqrt((p*(1-p) + z**2/(4*n))/n)) / denom
    return center, center - half, center + half

# ---------------------- Site list (robust) ----------------------
site_list_col = (find_col(sites_df,
                          ["Site Name","Site","site","site_name","SURVEY_NAM","survey_nam"],
                          contains="site")
                 or sites_df.columns[0])
SITE_LIST = (sites_df[site_list_col]
             .dropna()
             .astype(str).str.strip()
             .unique()
             .tolist())

# ---------------------- 1) Biology to monthly HO occupancy ----------------------
site_col  = find_col(bio, ["SURVEY_NAM","Site","site","site_name","survey_name"], contains="site")
year_col  = find_col(bio, ["YEAR","Year","year"])
month_col = find_col(bio, ["MN","Month","month"])
if site_col is None or year_col is None or month_col is None:
    raise KeyError("Cannot find site/year/month columns in biology table.")

bio_f = bio[bio[site_col].astype(str).isin(SITE_LIST)].copy()

# monthly datetime (first day of month)
month_num = parse_month_series(bio_f[month_col])
bio_f["date"] = pd.to_datetime(
    dict(year=bio_f[year_col].astype(int), month=month_num.astype(int), day=1),
    errors="coerce"
)
bio_f = bio_f.rename(columns={site_col: "SURVEY_NAM"}).dropna(subset=["date"])

# hard filter to 2010–2018
bio_f = bio_f[bio_f["date"].dt.year.between(YEAR_MIN, YEAR_MAX)]

# HO presence flag
hcol = find_col(bio_f,
                ["Halophila ovalis","H. ovalis","H_ovalis","H ovalis","HOVALIS","HO","ho"],
                contains="ovalis")
if hcol is None:
    raise KeyError("Cannot find the Halophila ovalis (HO) column in biology table.")

val = bio_f[hcol]
if np.issubdtype(val.dtype, np.number):
    bio_f["HO_present"] = val.fillna(0) > 0
else:
    s = val.astype(str).str.strip().str.lower()
    bio_f["HO_present"] = s.isin(["yes","present","1","true","y"])

# occupancy (n, k) + Wilson CI + continuity-corrected p_cc
occ_ho = (bio_f
          .groupby(["SURVEY_NAM","date"], as_index=False)
          .agg(n=("HO_present","size"), k=("HO_present","sum")))
occ_ho["p"], occ_ho["p_lo"], occ_ho["p_hi"] = wilson_ci(occ_ho["k"], occ_ho["n"])
occ_ho["p_lo"] = occ_ho["p_lo"].clip(lower=0)
occ_ho["p_hi"] = occ_ho["p_hi"].clip(upper=1)
occ_ho["p_cc"]  = (occ_ho["k"] + 0.5) / (occ_ho["n"] + 1.0)

# ---------------------- 2) Environment (pivot if long) ----------------------
env2 = env.copy()

# date column
date_col = ( "Aggregated Date/Time" if "Aggregated Date/Time" in env2.columns else
             (find_col(env2, ["date","Date","datetime","Datetime"], contains="date")) )
if date_col is None:
    raise KeyError("Cannot locate a datetime column in env table (expected 'Aggregated Date/Time' or 'date').")
env2["date"] = pd.to_datetime(env2[date_col], errors="coerce")

# site column
site_env_col = ( "Site Name" if "Site Name" in env2.columns else
                 (find_col(env2, ["SURVEY_NAM","Site","site","site_name"], contains="site")) )
if site_env_col is None:
    raise KeyError("Cannot find site column in env table (e.g. 'Site Name'/'site').")
env2 = env2.rename(columns={site_env_col: "SURVEY_NAM"}).dropna(subset=["date"])

# align to sites in biology and years 2010–2018
env2 = env2[env2["SURVEY_NAM"].astype(str).isin(occ_ho["SURVEY_NAM"].astype(str).unique())]
env2 = env2[env2["date"].dt.year.between(YEAR_MIN, YEAR_MAX)]

# pivot if long-form (Variable, mean), else keep as wide-form
if {"Variable","mean"}.issubset(env2.columns):
    env_w = (env2.pivot_table(index=["SURVEY_NAM","date"],
                              columns="Variable", values="mean", aggfunc="mean")
                  .reset_index())
    env_w.columns.name = None
else:
    env_w = env2.copy()

# derived speeds
if {"u","v"}.issubset(env_w.columns):
    env_w["current_speed"] = np.sqrt(env_w["u"]**2 + env_w["v"]**2)
if {"wspeed_u","wspeed_v"}.issubset(env_w.columns):
    env_w["wind_speed"] = np.sqrt(env_w["wspeed_u"]**2 + env_w["wspeed_v"]**2)

# ---------------------- 3) Merge + lags ----------------------
merged_ho = (occ_ho
             .merge(env_w, on=["SURVEY_NAM","date"], how="left")
             .sort_values(["SURVEY_NAM","date"]))

merged_ho["month"] = merged_ho["date"].dt.month

driver_cols = [c for c in ["temp","salt","eta","current_speed","wind_speed"] if c in merged_ho.columns]
for v in driver_cols:
    for L in (1,2,3):
        merged_ho[f"{v}_l{L}"] = merged_ho.groupby("SURVEY_NAM", sort=False)[v].shift(L)

merged_ho["p_cc_l1"] = merged_ho.groupby("SURVEY_NAM", sort=False)["p_cc"].shift(1)

# hard filter again (safety) to 2010–2018 after merge
merged_ho = merged_ho[merged_ho["date"].dt.year.between(YEAR_MIN, YEAR_MAX)]

# ---------------------- 4) Save ----------------------
occ_ho.sort_values(["SURVEY_NAM","date"]).to_csv(OUT / "Ho_monthly_occupancy_2010_2018.csv", index=False)
merged_ho.to_csv(OUT / "Ho_occ_with_env_2010_2018.csv", index=False)
print("Saved:\n  Ho_monthly_occupancy_2010_2018.csv\n  Ho_occ_with_env_2010_2018.csv\nFolder:", OUT.resolve())

Saved:
  Ho_monthly_occupancy_2010_2018.csv
  Ho_occ_with_env_2010_2018.csv
Folder: C:\Users\F
